In [5]:
import pywbemReq
c = pywbemReq.WBEMConnection('https://10.109.23.127:5989', ('Local/admin', 'Password123!'), default_namespace='root/emc/smis')

cifs_share = [share for share in
              c.EnumerateInstances('EMC_VNXe_CIFSShareLeaf', namespace='root/emc/smis')
              if share['InstanceID'] == 'SMBShare_1202'][0]

print(cifs_share.tomof())
setting = c.Associators(cifs_share.path, ResultClass='CIM_SettingData')[0]
print('{} {}'.format('*' * 5, 'Setting of CIFS Share:'))
print(setting.tomof())
print('-' * 50)

for p in c.References(cifs_share.path, ResultClass='EMC_VNXe_CIFSShare_Identity_AssociatedPrivilegeAssocLeaf'):
    print(p.tomof())


instance of EMC_VNXe_CIFSShareLeaf {
	RequestedState = 12;
	ElementName = "cycle33_file_attach_issue_debug";
	Description = "";
	TransitioningToState = 12;
	InstanceID = "SMBShare_1202";
	EnabledState = 5;
	SharingDirectory = True;
	EnabledDefault = 2;
	OperationalStatus = {2};
	PathNameSeparatorString = "\\";
	Name = "cycle33_file_attach_issue_debug";
};

***** Setting of CIFS Share:
instance of EMC_VNXe_CIFSShareSettingLeaf {
	WritePolicy = 0;
	CASupported = True;
	ElementName = "cycle33_file_attach_issue_debug";
	RootAccess = 3;
	InitialEnabledState = 2;
	InstanceID = "SMBShare_1202";
	ProtocolVersions = {"Latest"};
	DefaultReadWrite = 3;
	ExecuteSupport = 0;
	DefaultUserIdSupported = 2;
	FileSharingProtocol = 3;
	AccessPoints = 5;
	DefaultExecute = 0;
};

--------------------------------------------------
instance of EMC_VNXe_CIFSShare_Identity_AssociatedPrivilegeAssocLeaf {
	Activities = {2, 3, 5, 6, 7, 14};
	Target = //10.109.23.126/root/emc/smis:EMC_VNXe_CIFSShareLeaf.InstanceID

In [17]:
# **********
# BASIC PART
# **********

import pywbem

c = pywbem.WBEMConnection('https://10.141.45.128:5989', ('Local/admin', 'Password123!'), default_namespace='root/emc/smis')
topCS = c.EnumerateInstances('EMC_VNXe_StorageSystemLeaf', namespace='root/emc/smis')[0]
aclService = c.EnumerateInstances('CIM_AccountManagementService',namespace='root/emc/smis')[0]

cifs_server = c.EnumerateInstances('EMC_VNXe_CIFSServerLeaf',namespace='root/emc/smis')[0]
print('Using the CIFS server: {}'.format(cifs_server.tomof()))

cifs_share_service = c.Associators(cifs_server.path, ResultClass= 'CIM_FileExportService')[0]
print('Using the CIFS export service: {}'.format(cifs_share_service.tomof()))

cifs_share_capabilities = c.Associators(cifs_share_service.path,
                                        AssocClass='CIM_ElementCapabilities',
                                        ResultClass='CIM_ExportedFileShareCapabilities')[0]
print('Using CIFS expport capabilities: {}'.format(cifs_share_capabilities.tomof()))

print('*' * 80)

nfs_server = c.EnumerateInstances('EMC_VNXe_NFSServerLeaf',namespace='root/emc/smis')[0]
print('Using the NFS server: {}'.format(nfs_server.tomof()))

nfs_share_service = c.Associators(nfs_server.path, ResultClass= 'CIM_FileExportService')[0]
print('Using the NFS export service: {}'.format(nfs_share_service.tomof()))

nfs_share_capabilities = c.Associators(nfs_share_service.path,
                                        AssocClass='CIM_ElementCapabilities',
                                        ResultClass='CIM_ExportedFileShareCapabilities')[0]
print('Using NFS expport capabilities: {}'.format(nfs_share_capabilities.tomof()))

right_read, right_write, right_full = pywbem.Uint16(5), pywbem.Uint16(6), pywbem.Uint16(14)

default_no_access, default_ro, default_rw = pywbem.Uint16(2), pywbem.Uint16(3), pywbem.Uint16(4)


Using the CIFS server: instance of EMC_VNXe_CIFSServerLeaf {
	RequestedState = 12;
	Dedicated = {16};
	Name = "NAS_1";
	TransitioningToState = 12;
	EnabledState = 5;
	ElementName = "nas_1";
	HealthState = 5;
	OperationalStatus = {0};
	NameFormat = "Other";
	CreationClassName = "EMC_VNXe_CIFSServerLeaf";
	PrimaryStatus = 1;
	StatusDescriptions = {"Unknown"};
};

Using the CIFS export service: instance of EMC_VNXe_FileExportServiceLeaf {
	CreationClassName = "EMC_VNXe_FileExportServiceLeaf";
	ElementName = "cifs_6:File Export Service";
	SystemName = "cifs_6";
	Name = "cifs_6";
	SystemCreationClassName = "EMC_VNXe_CIFSServerLeaf";
};

Using CIFS expport capabilities: instance of EMC_VNXe_CIFSShareCapabilitiesLeaf {
	InstanceID = "cifs_6";
	CASupported = True;
	ElementName = "Exported File Share Capabilities:CIFS";
	SupportedProperties = {2, 4, 5, 7, 8};
	FileSharingProtocol = 3;
};

********************************************************************************
Using the NFS server: inst

In [72]:
# *****************
# CREATE CIFS SHARE
# *****************

cifs_share_name = 'cifs_5'
fs_name = 'fs_cifs_1'

fs = filter(lambda x: x['ElementName'] == fs_name, c.EnumerateInstances('CIM_FileSystem'))[0]
print('Using File System: {}'.format(fs.tomof()))

result = c.InvokeMethod('CreateGoalSettings', cifs_share_capabilities.path)
cifs_goalsettings = result[1]['SupportedGoalSettings'][0]
cifs_goalsettings['DefaultReadWrite'] = default_no_access
print('Using CIFS Goal Setting: {}'.format(cifs_goalsettings.tomof()))

result = c.InvokeMethod('SNIA_CreateExportedShare',
                        cifs_share_service.path,
                        ElementName=cifs_share_name,
                        Root=fs.path,
                        SharedElementPath='/',
                        Goal=cifs_goalsettings)
print('Result of CreateExportedShare: {}'.format(result[0]))
cifs_share = c.GetInstance(result[1]['TheShare'])
print('CIFS share created: {}'.format(cifs_share.tomof()))

Using File System: instance of EMC_VNXe_UxfsLocalFileSystemLeaf {
	MaxFileNameLength = 255;
	RequestedState = 12;
	TransitioningToState = 12;
	BlockSize = 1024;
	FileSystemType = "EMC_UXFS";
	IsFixedSize = 0;
	PathNameSeparatorString = "/";
	LocalAccessDefinitionRequired = 4;
	ResizeIncrement = 0;
	OperationalStatus = {2};
	EnabledDefault = 2;
	EnabledState = 5;
	CSCreationClassName = "EMC_VNXe_StorageSystemLeaf";
	FileSystemSize = 10737418240;
	CreationClassName = "EMC_VNXe_UxfsLocalFileSystemLeaf";
	ElementName = "fs_cifs_1";
	Name = "fs_11";
	CSName = "FCNCH097403344";
	NumberOfFiles = 0;
	CasePreserved = True;
	CaseSensitive = True;
	AvailableSpace = 9114959872;
};

Using CIFS Goal Setting: instance of EMC_VNXe_CIFSShareSettingLeaf {
	WritePolicy = 0;
	CASupported = True;
	ElementName = "Predefined FileServer Name cifs_6";
	RootAccess = 3;
	InitialEnabledState = 2;
	InstanceID = "cifs_6";
	ProtocolVersions = {"Latest"};
	DefaultReadWrite = 2;
	ExecuteSupport = 0;
	DefaultUserIdSupp

In [76]:
# *****************
# CREATE NFS SHARE
# *****************

nfs_share_name = 'nfs_1'
fs_name = 'fs_nfs_1'

fs = filter(lambda x: x['ElementName'] == fs_name, c.EnumerateInstances('CIM_FileSystem'))[0]
print('Using File System: {}'.format(fs.tomof()))

result = c.InvokeMethod('CreateGoalSettings', nfs_share_capabilities.path)
nfs_goalsettings = result[1]['SupportedGoalSettings'][0]
nfs_goalsettings['DefaultReadWrite'] = default_rw
print('Using NFS Goal Setting: {}'.format(nfs_goalsettings.tomof()))

result = c.InvokeMethod('SNIA_CreateExportedShare',
                        nfs_share_service.path,
                        ElementName=nfs_share_name,
                        Root=fs.path,
                        SharedElementPath='/',
                        Goal=nfs_goalsettings)
print('Result of CreateExportedShare: {}'.format(result[0]))
nfs_share = c.GetInstance(result[1]['TheShare'])
print('NFS share created: {}'.format(nfs_share.tomof()))

Using File System: instance of EMC_VNXe_UxfsLocalFileSystemLeaf {
	MaxFileNameLength = 255;
	RequestedState = 12;
	TransitioningToState = 12;
	BlockSize = 1024;
	FileSystemType = "EMC_UXFS";
	IsFixedSize = 0;
	PathNameSeparatorString = "/";
	LocalAccessDefinitionRequired = 4;
	ResizeIncrement = 0;
	OperationalStatus = {2};
	EnabledDefault = 2;
	EnabledState = 5;
	CSCreationClassName = "EMC_VNXe_StorageSystemLeaf";
	FileSystemSize = 10737418240;
	CreationClassName = "EMC_VNXe_UxfsLocalFileSystemLeaf";
	ElementName = "fs_nfs_1";
	Name = "fs_10";
	CSName = "FCNCH097403344";
	NumberOfFiles = 0;
	CasePreserved = True;
	CaseSensitive = True;
	AvailableSpace = 9117114368;
};

Using NFS Goal Setting: instance of EMC_VNXe_NFSShareSettingLeaf {
	WritePolicy = 0;
	InitialEnabledState = 2;
	ElementName = "Predefined FileServer Name nfs_7";
	RootAccess = 3;
	InstanceID = "nfs_7";
	ProtocolVersions = {"v2", "v3", "v4"};
	DefaultReadWrite = 4;
	ExecuteSupport = 0;
	DefaultUserIdSupported = 2;
	FileSh

In [77]:
# ***************
# LIST ALL SHARES
# ***************

cifs_share_list = c.EnumerateInstances('EMC_VNXe_CIFSShareLeaf', namespace='root/emc/smis')
print('All CIFS shares:')
for each in cifs_share_list:
    print(each.tomof())
    setting = c.Associators(each.path, ResultClass='CIM_SettingData')[0]
    print('{} {}'.format('*' * 5, 'Setting of CIFS Share:'))
    print(setting.tomof())
    print('-' * 50)

print("*" * 80)
    
nfs_share_list = c.EnumerateInstances('EMC_VNXe_NFSShareLeaf', namespace='root/emc/smis')
print('All NFS shares:')
for each in nfs_share_list:
    print(each.tomof())
    setting = c.Associators(each.path, ResultClass='CIM_SettingData')[0]
    print('{} {}'.format('*' * 5, 'Setting of NFS Share:'))
    print(setting.tomof())
    print('-' * 50)

All CIFS shares:
instance of EMC_VNXe_CIFSShareLeaf {
	RequestedState = 12;
	ElementName = "cifs_1";
	Description = "";
	TransitioningToState = 12;
	InstanceID = "SMBShare_11";
	EnabledState = 5;
	SharingDirectory = True;
	EnabledDefault = 2;
	OperationalStatus = {2};
	PathNameSeparatorString = "\\";
	Name = "cifs_1";
};

***** Setting of CIFS Share:
instance of EMC_VNXe_CIFSShareSettingLeaf {
	WritePolicy = 0;
	CASupported = True;
	ElementName = "cifs_1";
	RootAccess = 3;
	InitialEnabledState = 2;
	InstanceID = "SMBShare_11";
	ProtocolVersions = {"Latest"};
	DefaultReadWrite = 3;
	ExecuteSupport = 0;
	DefaultUserIdSupported = 2;
	FileSharingProtocol = 3;
	AccessPoints = 5;
	DefaultExecute = 0;
};

--------------------------------------------------
instance of EMC_VNXe_CIFSShareLeaf {
	RequestedState = 12;
	ElementName = "cifs_2";
	Description = "";
	TransitioningToState = 12;
	InstanceID = "SMBShare_12";
	EnabledState = 5;
	SharingDirectory = True;
	EnabledDefault = 2;
	OperationalSta

In [26]:
# *******************
# CREATE USER CONTACT
# *******************

userID = 'win2012.dev\\administrator'
contactClassName = "CIM_UserContact"
contact = pywbem.CIMInstance(contactClassName,{"Name" : userID, "CreationClassName" : contactClassName})
print('Using the User Contact: {}'.format(contact.tomof()))

result = c.InvokeMethod("CreateUserContact", aclService.path, System = topCS.path, UserContactTemplate = contact)
print('Result of CreateUserContact: {}'.format(result[0]))
identity = c.GetInstance(result[1]['Identities'][0])
print('User created: {}'.format(identity.tomof()))

Using the User Contact: instance of CIM_UserContact {
	CreationClassName = "CIM_UserContact";
	Name = "win2012.dev\administrator";
};

Result of CreateUserContact: 0
User created: instance of EMC_VNXe_IdentityLeaf {
	InstanceID = "S-1-5-15-8aa47157-8def95d8-5c9921c1-1f4";
	ElementName = "S-1-5-15-8aa47157-8def95d8-5c9921c1-1f4";
};



In [67]:
# ****************
# ASSIGN PRIVILEGE
# ****************

share_id = 'SMBShare_11'

cifs_share = filter(lambda x: x['InstanceID'] == share_id,
                    c.EnumerateInstances('EMC_VNXe_CIFSShareLeaf', namespace='root/emc/smis'))[0]
print('{} {}'.format('*' * 5, 'Check CIFS Share:'))
print(cifs_share.tomof())

identity = c.EnumerateInstances('EMC_VNXe_IdentityLeaf', namespace='root/emc/smis')[0]
print('{} Using the Identity: {}'.format('*' * 5, identity.tomof()))

result = c.InvokeMethod("AssignPrivilegeToExportedShare",
                        exportService.path,
                        Identities=[identity.path],
                        Activities=[right_write],
                        FileShare=cifs_share.path)
print('Result of AssignPrivilege: {}'.format(result[0]))


***** Check CIFS Share:
instance of EMC_VNXe_CIFSShareLeaf {
	RequestedState = 12;
	ElementName = "cifs_1";
	Description = "";
	TransitioningToState = 12;
	InstanceID = "SMBShare_11";
	EnabledState = 5;
	SharingDirectory = True;
	EnabledDefault = 2;
	OperationalStatus = {2};
	PathNameSeparatorString = "\\";
	Name = "cifs_1";
};

***** Using the Identity: instance of EMC_VNXe_IdentityLeaf {
	InstanceID = "S-1-5-15-8aa47157-8def95d8-5c9921c1-453";
	ElementName = "S-1-5-15-8aa47157-8def95d8-5c9921c1-453";
};

Result of AssignPrivilege: 0


In [35]:
# ****************
# MODIFY PRIVILEGE
# ****************

share_id = 'SMBShare_7'
sid_filter = '453'

cifs_share = filter(lambda x: x['InstanceID'] == share_id,
                    c.EnumerateInstances('EMC_VNXe_CIFSShareLeaf', namespace='root/emc/smis'))[0]
print('{} {}'.format('*' * 5, 'Check CIFS Share:'))
print(cifs_share.tomof())

privilege_list = c.References(cifs_share.path, ResultClass='EMC_VNXe_CIFSShare_Identity_AssociatedPrivilegeAssocLeaf')
privilege = [e for e in privilege_list if sid_filter in str(e['Subject'])][0]
print('*' * 80)
print('Before modification')
print(privilege.tomof())

privilege['Activities'] = [right_read]
c.ModifyInstance(privilege)

privilege_list = c.References(cifs_share.path, ResultClass='EMC_VNXe_CIFSShare_Identity_AssociatedPrivilegeAssocLeaf')
privilege = [e for e in privilege_list if sid_filter in str(e['Subject'])][0]
print('*' * 80)
print('After modification')
print(privilege.tomof())

***** Check CIFS Share:
instance of EMC_VNXe_CIFSShareLeaf {
	RequestedState = 12;
	ElementName = "cifs_1";
	Description = "Add comment from SMIS.";
	TransitioningToState = 12;
	InstanceID = "SMBShare_7";
	EnabledState = 5;
	SharingDirectory = True;
	EnabledDefault = 2;
	OperationalStatus = {2};
	PathNameSeparatorString = "\\";
	Name = "cifs_1";
};

********************************************************************************
Before modification
instance of EMC_VNXe_CIFSShare_Identity_AssociatedPrivilegeAssocLeaf {
	Activities = {2, 3, 5, 6, 7};
	Target = //10.141.45.128/root/emc/smis:EMC_VNXe_CIFSShareLeaf.InstanceID="SMBShare_7";
	UseKey = "Default";
	RepresentsAuthorizationRights = False;
	PrivilegeGranted = True;
	Subject = //10.141.45.128/root/emc/smis:EMC_VNXe_IdentityLeaf.InstanceID="S-1-5-15-8aa47157-8def95d8-5c9921c1-453";
};

********************************************************************************
After modification
instance of EMC_VNXe_CIFSShare_Identity_Associate

In [63]:
# ****************
# DELETE PRIVILEGE
# ****************

share_id = 'SMBShare_10'
sid_filter = '453'

cifs_share = filter(lambda x: x['InstanceID'] == share_id,
                    c.EnumerateInstances('EMC_VNXe_CIFSShareLeaf', namespace='root/emc/smis'))[0]
print('{} {}'.format('*' * 5, 'Check CIFS Share:'))
print(cifs_share.tomof())

privilege_list = c.References(cifs_share.path, ResultClass='EMC_VNXe_CIFSShare_Identity_AssociatedPrivilegeAssocLeaf')
privilege = [e for e in privilege_list if sid_filter in str(e['Subject'])][0]
print('*' * 80)
print('Before modification')
print(privilege.tomof())

c.DeleteInstance(privilege.path)

privilege_list = c.References(cifs_share.path, ResultClass='EMC_VNXe_CIFSShare_Identity_AssociatedPrivilegeAssocLeaf')
privilege = [e for e in privilege_list if sid_filter in str(e['Subject'])][0]
print('*' * 80)
print('After modification')
print(privilege.tomof())

***** Check CIFS Share:
instance of EMC_VNXe_CIFSShareLeaf {
	RequestedState = 12;
	ElementName = "cifs_3";
	Description = "Add comment from SMIS.";
	TransitioningToState = 12;
	InstanceID = "SMBShare_10";
	EnabledState = 5;
	SharingDirectory = True;
	EnabledDefault = 2;
	OperationalStatus = {2};
	PathNameSeparatorString = "\\";
	Name = "cifs_3";
};

********************************************************************************
Before modification
instance of EMC_VNXe_CIFSShare_Identity_AssociatedPrivilegeAssocLeaf {
	Activities = {2, 3, 5, 6, 7};
	Target = //10.141.45.128/root/emc/smis:EMC_VNXe_CIFSShareLeaf.InstanceID="SMBShare_10";
	UseKey = "Default";
	RepresentsAuthorizationRights = False;
	PrivilegeGranted = True;
	Subject = //10.141.45.128/root/emc/smis:EMC_VNXe_IdentityLeaf.InstanceID="S-1-5-15-8aa47157-8def95d8-5c9921c1-453";
};



IndexError: list index out of range

In [68]:
# **************
# LIST PRIVILEGE
# **************

share_id = 'SMBShare_11'

for e in c.EnumerateInstances('EMC_VNXe_CIFSShareLeaf', namespace='root/emc/smis'):
    if share_id is not None and e['InstanceID'] != share_id:
        continue
    print('=' * 50)
    print('{} {}'.format('*' * 5, 'Check CIFS Share:'))
    print(e.tomof())
    print('-' * 50)
    
    cifs_setting = c.Associators(e.path, ResultClass='CIM_SettingData')[0]
    print('{} {}'.format('*' * 5, 'Setting of CIFS Share:'))
    print(cifs_setting.tomof())
    print('-' * 50)

    print('{} {}'.format('*' * 5, 'ACE of CIFS Share:'))
    for e2 in c.References(e.path, ResultClass='EMC_VNXe_CIFSShare_Identity_AssociatedPrivilegeAssocLeaf'):
        print(e2.tomof())

***** Check CIFS Share:
instance of EMC_VNXe_CIFSShareLeaf {
	RequestedState = 12;
	ElementName = "cifs_1";
	Description = "";
	TransitioningToState = 12;
	InstanceID = "SMBShare_11";
	EnabledState = 5;
	SharingDirectory = True;
	EnabledDefault = 2;
	OperationalStatus = {2};
	PathNameSeparatorString = "\\";
	Name = "cifs_1";
};

--------------------------------------------------
***** Setting of CIFS Share:
instance of EMC_VNXe_CIFSShareSettingLeaf {
	WritePolicy = 0;
	CASupported = True;
	ElementName = "cifs_1";
	RootAccess = 3;
	InitialEnabledState = 2;
	InstanceID = "SMBShare_11";
	ProtocolVersions = {"Latest"};
	DefaultReadWrite = 3;
	ExecuteSupport = 0;
	DefaultUserIdSupported = 2;
	FileSharingProtocol = 3;
	AccessPoints = 5;
	DefaultExecute = 0;
};

--------------------------------------------------
***** ACE of CIFS Share:
instance of EMC_VNXe_CIFSShare_Identity_AssociatedPrivilegeAssocLeaf {
	Activities = {2, 3, 5, 6, 7};
	Target = //10.141.45.128/root/emc/smis:EMC_VNXe_CIFSSha

In [60]:
# *****************
# MODIFY CIFS SHARE
# *****************

share_id = 'SMBShare_10'
cifs_share = filter(lambda x: x['InstanceID'] == share_id,
                    c.EnumerateInstances('EMC_VNXe_CIFSShareLeaf', namespace='root/emc/smis'))[0]
print('{} {}'.format('*' * 5, 'Check CIFS Share:'))
print(cifs_share.tomof())

result = c.InvokeMethod('CreateGoalSettings', cifs_share_capabilities.path)
modified_setting = result[1]['SupportedGoalSettings'][0]
modified_setting['DefaultReadWrite'] = default_ro
print('Using CIFS Goal Setting: {}'.format(cifs_goalsettings.tomof()))

result = c.InvokeMethod('SNIA_ModifyExportedShare',
                        export_service.path,
                        TheShare=cifs_share.path,
                        Comment='Add comment from SMIS.')
                        #Comment='Modify DefaultReadWrite to WRITE',
                        #Goal=modified_setting)
print('SNIA_ModifyExportedShare result: {}'.format(result[0]))


***** Check CIFS Share:
instance of EMC_VNXe_CIFSShareLeaf {
	RequestedState = 12;
	ElementName = "cifs_3";
	Description = "";
	TransitioningToState = 12;
	InstanceID = "SMBShare_10";
	EnabledState = 5;
	SharingDirectory = True;
	EnabledDefault = 2;
	OperationalStatus = {2};
	PathNameSeparatorString = "\\";
	Name = "cifs_3";
};

Using CIFS Goal Setting: instance of EMC_VNXe_CIFSShareSettingLeaf {
	WritePolicy = 0;
	CASupported = True;
	ElementName = "Predefined FileServer Name cifs_6";
	RootAccess = 3;
	InitialEnabledState = 2;
	InstanceID = "cifs_6";
	ProtocolVersions = {"Latest"};
	DefaultReadWrite = 2;
	ExecuteSupport = 0;
	DefaultUserIdSupported = 2;
	FileSharingProtocol = 3;
	AccessPoints = 5;
	DefaultExecute = 0;
};

SNIA_ModifyExportedShare result: 0


In [53]:
# ****************
# MODIFY NFS SHARE
# ****************

share_id = 'NFSShare_4'
nfs_share = filter(lambda x: x['InstanceID'] == share_id,
                   c.EnumerateInstances('EMC_VNXe_NFSShareLeaf', namespace='root/emc/smis'))[0]
print('{} {}'.format('*' * 5, 'Check NFS Share:'))
print(nfs_share.tomof())

result = c.InvokeMethod('CreateGoalSettings', nfs_share_capabilities.path)
modified_setting = result[1]['SupportedGoalSettings'][0]
modified_setting['DefaultReadWrite'] = default_ro
print('Using NFS Goal Setting: {}'.format(nfs_goalsettings.tomof()))

result = c.InvokeMethod('SNIA_ModifyExportedShare',
                        nfs_share_service.path,
                        TheShare=nfs_share.path,
                        Comment='Modify DefaultReadWrite to READ',
                        Goal=modified_setting)
print('SNIA_ModifyExportedShare result: {}'.format(result[0]))

***** Check NFS Share:
instance of EMC_VNXe_NFSShareLeaf {
	RequestedState = 12;
	ElementName = "nfs_snap_1";
	Name = "nfs_snap_1";
	TransitioningToState = 12;
	InstanceID = "NFSShare_4";
	EnabledState = 5;
	SharingDirectory = True;
	EnabledDefault = 2;
	OperationalStatus = {2};
	PathNameSeparatorString = "/";
};

Using NFS Goal Setting: instance of EMC_VNXe_NFSShareSettingLeaf {
	WritePolicy = 0;
	InitialEnabledState = 2;
	ElementName = "Predefined FileServer Name nfs_7";
	RootAccess = 3;
	InstanceID = "nfs_7";
	ProtocolVersions = {"v2", "v3", "v4"};
	DefaultReadWrite = 4;
	ExecuteSupport = 0;
	DefaultUserIdSupported = 2;
	FileSharingProtocol = 2;
	AccessPoints = 4;
	DefaultExecute = 0;
};

SNIA_ModifyExportedShare result: 0


In [54]:
# *****************
# DELETE CIFS SHARE
# *****************

share_id = 'SMBShare_9'
cifs_share = filter(lambda x: x['InstanceID'] == share_id,
                    c.EnumerateInstances('EMC_VNXe_CIFSShareLeaf', namespace='root/emc/smis'))[0]
print('{} {}'.format('*' * 5, 'Check CIFS Share:'))
print(cifs_share.tomof())

result = c.InvokeMethod('ReleaseExportedShare',
                        cifs_share_service.path,
                        TheShare=cifs_share.path)
print('SNIA_ModifyExportedShare result: {}'.format(result[0]))

***** Check CIFS Share:
instance of EMC_VNXe_CIFSShareLeaf {
	RequestedState = 12;
	ElementName = "cifs_snap_1";
	Description = "Modify DefaultReadWrite to WRITE";
	TransitioningToState = 12;
	InstanceID = "SMBShare_9";
	EnabledState = 5;
	SharingDirectory = True;
	EnabledDefault = 2;
	OperationalStatus = {2};
	PathNameSeparatorString = "\\";
	Name = "cifs_snap_1";
};

SNIA_ModifyExportedShare result: 0


In [75]:
# *****************
# DELETE NFS SHARE
# *****************

share_id = 'NFSShare_5'
nfs_share = filter(lambda x: x['InstanceID'] == share_id,
                   c.EnumerateInstances('EMC_VNXe_NFSShareLeaf', namespace='root/emc/smis'))[0]
print('{} {}'.format('*' * 5, 'Check NFS Share:'))
print(nfs_share.tomof())

result = c.InvokeMethod('ReleaseExportedShare',
                        nfs_share_service.path,
                        TheShare=nfs_share.path)
print('SNIA_ModifyExportedShare result: {}'.format(result[0]))

***** Check NFS Share:
instance of EMC_VNXe_NFSShareLeaf {
	RequestedState = 12;
	ElementName = "nfs_1";
	Name = "nfs_1";
	TransitioningToState = 12;
	InstanceID = "NFSShare_5";
	EnabledState = 5;
	SharingDirectory = True;
	EnabledDefault = 2;
	OperationalStatus = {2};
	PathNameSeparatorString = "/";
};

SNIA_ModifyExportedShare result: 0


In [48]:
uisShare = c.EnumerateInstances('EMC_UIS_CIFSShare', namespace='root/emc')[1]
print("UIS Share: {}".format(uisShare.tomof()))
uisHostService = c.EnumerateInstances('EMC_UIS_HostServices', namespace='root/emc')[0]
result = c.InvokeMethod('GetACEsForShare', uisHostService.path, CIFSShare=uisShare.path)
print('Result of UIS GetACEsForShare: {}'.format(result[0]))
for e in result[1]['shareACEs']:
    print(e.tomof())

UIS Share: instance of C4CB_UIS_CIFSShare {
	LastModifyDate = 20160929144828.503602+000;
	ParentApplication_FriendlyID = "res_33";
	Umask = "022";
	Description = "test cifs ...";
	InstallDate = 20160929082414.560796+000;
	InstanceID = "root/emc:EMC_UEM_CIFSShareLeaf%InstanceID=SMBShare_6";
	Encryption = False;
	FriendlyID = "SMBShare_6";
	ParentFilesystem = "root/emc:EMC_UEM_FileSystemLeaf%InstanceID=fs_9";
	DFS = False;
	Name = "cifs_1";
	OfflineFlag = 3;
	BranchCache = False;
	ParentFilesystem_FriendlyID = "fs_9";
	IsACEEnabled = True;
	ExportPaths = {"\\nas_1.win2012.dev\cifs_1", "\\10.141.45.129\cifs_1"};
	ContinuousAvailability = True;
	ElementType = "com.emc.uem.cifsshare.sharedfolder";
	Path = "/";
	ABE = False;
};

Result of UIS GetACEsForShare: 0
instance of C4CB_UIS_CIFSShareACE {
	InstanceID = "C4CB_UIS_CIFSShareACE";
	AccessLevel = 2;
	AccessType = 1;
	SID = "S-1-1-0";
};

instance of C4CB_UIS_CIFSShareACE {
	InstanceID = "C4CB_UIS_CIFSShareACE";
	AccessLevel = 2;
	AccessTy

In [48]:
for e in c.EnumerateInstances('CIM_FileSystem'):
    print(e.tomof())

instance of EMC_VNXe_UxfsLocalFileSystemLeaf {
	MaxFileNameLength = 255;
	RequestedState = 12;
	TransitioningToState = 12;
	BlockSize = 1024;
	FileSystemType = "EMC_UXFS";
	IsFixedSize = 0;
	PathNameSeparatorString = "/";
	LocalAccessDefinitionRequired = 4;
	ResizeIncrement = 0;
	OperationalStatus = {2};
	EnabledDefault = 2;
	EnabledState = 5;
	CSCreationClassName = "EMC_VNXe_StorageSystemLeaf";
	FileSystemSize = 10737418240;
	CreationClassName = "EMC_VNXe_UxfsLocalFileSystemLeaf";
	ElementName = "fs_1";
	Name = "fs_9";
	CSName = "FCNCH097403344";
	NumberOfFiles = 0;
	CasePreserved = True;
	CaseSensitive = True;
	AvailableSpace = 9114959872;
};



In [48]:
for e in c.EnumerateInstances('CIM_FileSystem'):
    print(e.tomof())

instance of EMC_VNXe_UxfsLocalFileSystemLeaf {
	MaxFileNameLength = 255;
	RequestedState = 12;
	TransitioningToState = 12;
	BlockSize = 1024;
	FileSystemType = "EMC_UXFS";
	IsFixedSize = 0;
	PathNameSeparatorString = "/";
	LocalAccessDefinitionRequired = 4;
	ResizeIncrement = 0;
	OperationalStatus = {2};
	EnabledDefault = 2;
	EnabledState = 5;
	CSCreationClassName = "EMC_VNXe_StorageSystemLeaf";
	FileSystemSize = 10737418240;
	CreationClassName = "EMC_VNXe_UxfsLocalFileSystemLeaf";
	ElementName = "fs_1";
	Name = "fs_9";
	CSName = "FCNCH097403344";
	NumberOfFiles = 0;
	CasePreserved = True;
	CaseSensitive = True;
	AvailableSpace = 9114959872;
};

